In [1]:
#r "nuget: Plotly.NET, 3.0.1"
#r "nuget: Plotly.NET.Interactive, 3.0.1"
#r "nuget: FSharp.Stats, 0.4.3"

Installed Packages FSharp.Stats, 0.4.3 Plotly.NET, 2.0.0-preview.16 Plotly.NET.Interactive, 2.0.0-preview.16

Loading extensions from `Plotly.NET.Interactive.dll`

In [2]:
open Plotly.NET
open Plotly.NET.ConfigObjects
open FSharp.Stats.Interpolation
open FSharp.Stats

### Defining Parameters:

In [3]:
let totalClvSupply = 45_000_000_000UL;
let totalYears = 1
let totalDays = totalYears * 365
let totalEpochs = totalDays / 5

### Helper Methods:

In [4]:
let rec diff (xs: list<int * float>) = 
    let sorted = xs |> List.sortBy (fun (i, _) -> i)
    match sorted with
        | [] -> []
        | [x] -> []
        | x::tail -> (fst x, (snd << List.head) tail - snd x) :: diff tail

let clvTokenDistributed xs: list<float> =
    xs 
    |> List.fold (fun acc a -> float (a + List.head acc):: acc) [0] 
    |> List.rev

let clvTokenDistribution (numEpochs: int) = 
    let numEpochsDoubled = numEpochs * 2;
    let xData = vector [0;numEpochs;numEpochsDoubled];
    let yData = vector [0.;1.;0.];
    let coefficients = Interpolation.Polynomial.coefficients xData yData 
    let interpolFunction x = Interpolation.Polynomial.fit coefficients x
    let fit = [0 .. numEpochs] |> List.map (fun x -> x,interpolFunction x)
    fit 
    |> diff

clvTokenDistribution totalEpochs
|> Chart.Line

<!-- Plotly chart will be drawn inside this DIV -->

#### 1-year Distribution

#### CLV Distribution

In [5]:
let numEpochs = totalEpochs;
let councilShare = 0.3;
let delegatorsShare = 0.65;
let nftDelegatorShare = 0.05;

let totalClvPerEpoch = 
    clvTokenDistribution numEpochs
    |> List.map (fun (_, v) -> (float totalClvSupply) * v)
    |> clvTokenDistributed

totalClvPerEpoch
|> List.zip [0..numEpochs]
|> Chart.Line

let councilClvPerEpoch = 
    totalClvPerEpoch
    |> List.map (fun v -> v * councilShare)
    |> List.zip [0..numEpochs]

let delegatorClvPerEpoch = 
    totalClvPerEpoch
    |> List.map (fun v -> v * delegatorsShare)
    |> List.zip [0..numEpochs]

let nftDelegatorClvPerEpoch = 
    totalClvPerEpoch
    |> List.map (fun v -> v * nftDelegatorShare)
    |> List.zip [0..numEpochs]


In [6]:
let totalClvDistChart =
    totalClvPerEpoch
    |> List.zip [0..numEpochs]
    |> Chart.Line
    |> Chart.withTraceName "Total CLV distributed"

let councilClvDistChart = 
    councilClvPerEpoch
    |> Chart.Line
    |> Chart.withTraceName "Council Members Share"

let delegatorsClvDistChart = 
    delegatorClvPerEpoch
    |> Chart.Line
    |> Chart.withTraceName "Delegators"

let nftDelegatorsClvDistChart = 
    nftDelegatorClvPerEpoch
    |> Chart.Line
    |> Chart.withTraceName "NFT Stakers"


let chartConfig = Config.init(StaticPlot=false, Autosizable=true)
[
    totalClvDistChart
    councilClvDistChart
    delegatorsClvDistChart
    nftDelegatorsClvDistChart
]
|> Chart.combine
|> Chart.withSize(Width=1200)
|> Chart.withTitle "Total CLV Tokens Distributed as a function of Epoch"


<!-- Plotly chart will be drawn inside this DIV -->

#### ADA Share Based on CLV

In [7]:
let councilMemberCount = 10;
let poolMargin = 3;
let saturation = 1.;
let aveAdaRewards = (340. + 1072. * saturation) * float councilMemberCount;
let aveDelegatorCount = 7000;

let adaRewardsPerEpoch = List.map (fun x -> aveAdaRewards * float x) [0..totalEpochs]

In [8]:
let councilAdaSharePerEpoch = 
    totalClvPerEpoch
    |> List.zip adaRewardsPerEpoch
    |> List.map (fun (ada, _) -> (float ada) * councilShare / (float councilMemberCount))
    |> List.zip [0..totalEpochs]
    |> Chart.Line
    |> Chart.withTraceName "Share per Council Member"

let delegatorAdaSharePerEpoch = 
    totalClvPerEpoch
    |> List.zip adaRewardsPerEpoch
    |> List.map (fun (ada, _) -> (float ada) * delegatorsShare / (float aveDelegatorCount * float councilMemberCount))
    |> List.zip [0..totalEpochs]
    |> Chart.Line
    |> Chart.withTraceName "Delegators Share (if shared evenly)"

let w1DelegatorAdaSharePerEpoch = 
    totalClvPerEpoch
    |> List.zip adaRewardsPerEpoch
    |> List.map (fun (ada, _) -> (float ada) * 0.01)
    |> List.zip [0..totalEpochs]
    |> Chart.Line
    |> Chart.withTraceName "1% Whale Share"

let w5DelegatorAdaSharePerEpoch = 
    totalClvPerEpoch
    |> List.zip adaRewardsPerEpoch
    |> List.map (fun (ada, _) -> (float ada) * 0.05)
    |> List.zip [0..totalEpochs]
    |> Chart.Line
    |> Chart.withTraceName "5% Whale Share"

let w10DelegatorAdaSharePerEpoch = 
    totalClvPerEpoch
    |> List.zip adaRewardsPerEpoch
    |> List.map (fun (ada, _) -> (float ada) * 0.1)
    |> List.zip [0..totalEpochs]
    |> Chart.Line
    |> Chart.withTraceName "10% Whale Share"

let nftDelegatorAdaSharePerEpoch = 
    totalClvPerEpoch
    |> List.zip adaRewardsPerEpoch
    |> List.map (fun (ada, _) -> (float ada) * nftDelegatorShare)
    |> List.zip [0..totalEpochs]
    |> Chart.Line
    |> Chart.withTraceName "NFT delegators Share"


let chartConfig = Config.init(StaticPlot=false, Autosizable=true)
[
    councilAdaSharePerEpoch
    delegatorAdaSharePerEpoch
    w1DelegatorAdaSharePerEpoch
    w5DelegatorAdaSharePerEpoch
    w10DelegatorAdaSharePerEpoch
    // nftDelegatorAdaSharePerEpoch
]
|> Chart.combine
|> Chart.withSize(Width=1200)
|> Chart.withTitle $"Total Ada Distributed as a function of Epoch ({councilMemberCount} Stake pools, {saturation * 100.} percent saturation across pools at {poolMargin} percent margin).";

<!-- Plotly chart will be drawn inside this DIV -->